In [1]:
import os

from openai import OpenAI, OpenAIError

In [2]:
# Global variables

# Debug flags
DEBUG = True

# LLM settings
TEMP = 1

# Huggingface
HUGGINGFACE_API_KEY = 

# OpenAI
OPENAI_API_KEY = 
OPENAI_ORG = 
OPENAI_PROJECT = 

In [3]:
# Send OpenAI request
def request_openai(system_prompt, prompt, model="gpt-4", n=1):
    client = OpenAI(
        api_key=OPENAI_API_KEY
    )

    if system_prompt:
        messages = [
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": prompt
            }
        ]
    else:
        messages = [
            {
                "role": "user",
                "content": prompt
            }
        ]

    # Send a request to OpenAI with the specified prompt
    DEBUG and print("Started requesting LLM")

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=TEMP,
        n=n
    )

    return response

In [4]:
# Save each completion in a separate .yaml file
def save_response_LLM(output_dir, response):
    for i, completion in enumerate(response.choices):
        content = completion.message.content
        
        # Ensure output folder exists
        os.makedirs(output_dir, exist_ok=True)

        file_path = output_dir + "/response-" + str(i) + ".yaml"
        
        # Write each response to a separate .yaml file
        with open(file_path, "w") as file:
            file.write(content)

        DEBUG and print(f"Saved LLM response to: {file_path}")

In [5]:
# Send second prompt to the same conversation (for CoT) and save the response
def second_request_openai(output_dir, prompt1, prompt2, intermediate_responses, model="gpt-4"):
    for i, completion in enumerate(intermediate_responses.choices):
        content = completion.message.content

        # Ensure output folder exists
        os.makedirs(output_dir, exist_ok=True)

        os.makedirs(output_dir + "/intermediate", exist_ok=True)

        file_path_intermediate = output_dir + "/intermediate" + "/response-" + str(i) + ".yaml"
        
        # Write each intermediate response to a separate .yaml file
        with open(file_path_intermediate, "w") as file:
            file.write(content)

        # Send second request with the conversation context
        client = OpenAI(
            api_key=OPENAI_API_KEY
        )

        messages = [
            {
                "role": "user",
                "content": prompt1
            },
            {
                "role": "assistant",
                "content": content # Reasoning response (from prompt1)
            },
            {
                "role": "user",
                "content": prompt2
            }
        ]

        # Send a request to OpenAI with the specified prompt
        DEBUG and print(f"Requesting second time LLM for response: {i}")

        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=TEMP,
            n=1
        )

        file_path = output_dir + "/response-" + str(i) + ".yaml"

        # Write each response to a separate .yaml file
        with open(file_path, "w") as file:
            file.write(response.choices[0].message.content)

        DEBUG and print(f"Saved LLM response to: {file_path}")

In [6]:
# Send meta enchanced prompt (for Meta prompting for prompting tasks) and save the response
def meta_request_openai(output_dir, system_prompt, intermediate_responses, model="gpt-4"):
    for i, completion in enumerate(intermediate_responses.choices):
        content = completion.message.content

        # Ensure output folder exists
        os.makedirs(output_dir, exist_ok=True)

        os.makedirs(output_dir + "/enhanced_prompts", exist_ok=True)

        file_path_intermediate = output_dir + "/enhanced_prompts" + "/response-" + str(i) + ".yaml"
        
        # Write each enhanced prompt response to a separate .yaml file
        with open(file_path_intermediate, "w") as file:
            file.write(content)

        # Send second request with the conversation context
        client = OpenAI(
            api_key=OPENAI_API_KEY
        )

        messages = [
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": content # Enhanced prompt (from meta prompting)
            }
        ]

        # Send a request to OpenAI with the specified prompt
        DEBUG and print(f"Requesting second time LLM for response: {i}")

        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=TEMP,
            n=1
        )

        file_path = output_dir + "/response-" + str(i) + ".yaml"

        # Write each response to a separate .yaml file
        with open(file_path, "w") as file:
            file.write(response.choices[0].message.content)

        DEBUG and print(f"Saved LLM response to: {file_path}")

# Prompts

In [7]:
# Baseline system prompt
bl_system_prompt = """Generate valid Kubernetes YAML manifests based on the provided information. Always output raw YAML only, wrapped within ```yaml blocks for formatting."""

# Persona system prompt
role_system_prompt = """You are an expert Kubernetes YAML generator, that only generates valid Kubernetes YAML manifests. You should never provide any explanations. You should always output raw YAML only, wrapped within ```yaml blocks for formatting."""

# Role-based best practices system prompt
role_best_system_prompt = """You are an expert Kubernetes YAML generator, that only generates valid Kubernetes YAML manifests, that adhere to Kubernetes best practices. You should never provide any explanations. You should always output raw YAML only, wrapped within ```yaml blocks for formatting."""

# Baseline user prompt
bl_user_prompt = """Create a Kubernetes YAML manifest to deploy a WordPress website using a MySQL database as the backend. Name the WordPress container "wordpress" and the MySQL container "mysql"."""

# Baseline detailed user prompt
bl_detailed_user_prompt = """
Create a Kubernetes YAML manifest to deploy a WordPress website with a MySQL database as the backend. Ensure the manifest includes the following:
	- Name the WordPress container "wordpress" and the MySQL container "mysql".
	- A Deployment for WordPress with appropriate container images, ports, and environment variables for database configuration.
	- A Deployment for MySQL, configured with PersistentVolumeClaims for data persistence.
	- Services for both WordPress and MySQL to enable connectivity.
	- Use ConfigMaps and Secrets for configuration and sensitive data.
"""

# Zero-Shot CoT reasoning prompts
reasoning_prompt = """Create a Kubernetes YAML manifest to deploy a WordPress website using a MySQL database as the backend. Name the WordPress container "wordpress" and the MySQL container "mysql". Let's think step by step."""
d_reasoning_prompt = """
Create a Kubernetes YAML manifest to deploy a WordPress website with a MySQL database as the backend. Ensure the manifest includes the following:
	- Name the WordPress container "wordpress" and the MySQL container "mysql".
	- A Deployment for WordPress with appropriate container images, ports, and environment variables for database configuration.
	- A Deployment for MySQL, configured with PersistentVolumeClaims for data persistence.
	- Services for both WordPress and MySQL to enable connectivity.
	- Use ConfigMaps and Secrets for configuration and sensitive data.
Let's think step by step.
"""
ape_reasoning_prompt = """Create a Kubernetes YAML manifest to deploy a WordPress website using a MySQL database as the backend. Name the WordPress container "wordpress" and the MySQL container "mysql". Let's work this out in a step-by-step way to be sure we have the right answer."""
ape_d_reasoning_prompt = """
Create a Kubernetes YAML manifest to deploy a WordPress website with a MySQL database as the backend. Ensure the manifest includes the following:
	- Name the WordPress container "wordpress" and the MySQL container "mysql".
	- A Deployment for WordPress with appropriate container images, ports, and environment variables for database configuration.
	- A Deployment for MySQL, configured with PersistentVolumeClaims for data persistence.
	- Services for both WordPress and MySQL to enable connectivity.
	- Use ConfigMaps and Secrets for configuration and sensitive data.
Let's work this out in a step-by-step way to be sure we have the right answer.
"""

# Zero-Shot CoT extraction prompt
extraction_prompt = """Based on your reasoning, generate a single YAML file containing the final answer. Ensure the YAML file is wrapped within ```yaml blocks for formatting, and follows proper YAML syntax."""

# Meta prompt
meta_system_prompt = """
You are an expert Kubernetes YAML generator, with specialized expertise in Kubernetes and YAML. Your core strengths lies in creating accurate, valid, and production-ready Kubernetes manifests tailored to specific requirements.

Your primary directive is to generate valid Kubernetes YAML manifests based on natural language descriptions, that adhere to Kubernetes best practices. This involves:

1. Analyzing the requirements to identify the necessary Kubernetes resources (Deployments, Services, ConfigMaps, Secrets, etc.).
2. Determining the appropriate YAML structure for each resource, including the kind, metadata, spec, and other relevant fields.
3. Apply your knowledge of Kubernetes to configure each resource, according to best practices.
4. Set up proper dependencies and relationships between resources, ensuring correct labels, selectors, metadata and configuration.
5. Combine all the configured resources into a comprehensive YAML manifest. Ensuring the YAML manifest is syntactically correct and ready for deployment without requiring further modification.
6. Output the generated YAML manifest, wrapped in yaml blocks for formatting, without providing explanations or additional text. 

Your proficiency in Kubernetes ensures that you can handle a wide range of scenarios, from simple deployments to complex multi-resource configurations.
"""

# Meta prompt for designing prompts with better reasoning capabilities.
meta_meta_prompt = """
Task: Prompt Revision to Enhance Reasoning Capabilities.
1. Input Prompt: "Create a Kubernetes YAML manifest to deploy a WordPress website using a MySQL database as the backend. Name the WordPress container "wordpress" and the MySQL container "mysql"."
2. Objective: Revise the above input prompt to enhance critical thinking and reasoning capabilities.
3. Key Elements for Revision:
	- Integrate complex problem-solving elements.
	- Embed multi-step reasoning processes.
	- Incorporate scenarios challenging conventional thinking.
4. Expected Outcome:
	- The revised prompt ([revised prompt]) should stimulate deeper analytical thought.
	- It should facilitate a comprehensive understanding of the subject matter.
	- Ensure the revised prompt fosters the exploration of diverse perspectives.
	- The prompt should encourage synthesis of information from various domains.
"""
d_meta_meta_prompt = """
Task: Prompt Revision to Enhance Reasoning Capabilities.
1. Input Prompt:
    "Create a Kubernetes YAML manifest to deploy a WordPress website with a MySQL database as the backend. Ensure the manifest includes the following:
	    - Name the WordPress container "wordpress" and the MySQL container "mysql".
	    - A Deployment for WordPress with appropriate container images, ports, and environment variables for database configuration.
	    - A Deployment for MySQL, configured with PersistentVolumeClaims for data persistence.
	    - Services for both WordPress and MySQL to enable connectivity.
	    - Use ConfigMaps and Secrets for configuration and sensitive data."
2. Objective: Revise the above input prompt to enhance critical thinking and reasoning capabilities.
3. Key Elements for Revision:
	- Integrate complex problem-solving elements.
	- Embed multi-step reasoning processes.
	- Incorporate scenarios challenging conventional thinking.
4. Expected Outcome:
	- The revised prompt ([revised prompt]) should stimulate deeper analytical thought.
	- It should facilitate a comprehensive understanding of the subject matter.
	- Ensure the revised prompt fosters the exploration of diverse perspectives.
	- The prompt should encourage synthesis of information from various domains.
"""

# ToT promts
tot_prompt_1 = """Imagine three different Kubernetes experts are answering this question. All experts will write down 1 step of their thinking, then share it with the group. Then all experts will go on to the next step, etc. If any expert realises they're wrong at any point then they leave. The question is: Create a Kubernetes YAML manifest to deploy a WordPress website using a MySQL database as the backend. Name the WordPress container "wordpress" and the MySQL container "mysql"."""
tot_prompt_2 = """Simulate three brilliant, Kubernetes experts collaboratively answering a question. Each one verbosely explains their thought process in real-time, considering the prior explanations of others and openly acknowledging mistakes. At each step, whenever possible, each expert refines and builds upon the thoughts of others, acknowledging their contributions. They continue until there is a definitive answer to the question. The question is: Create a Kubernetes YAML manifest to deploy a WordPress website using a MySQL database as the backend. Name the WordPress container "wordpress" and the MySQL container "mysql"."""
tot_prompt_3 = """Identify and behave as three different experts that are appropriate to answering this question. All experts will write down the step and their thinking about the step, then share it with the group. Then, all experts will go on to the next step, etc. At each step all experts will score their peers response between 1 and 5, 1 meaning it is highly unlikely, and 5 meaning it is highly likely. If any expert is judged to be wrong at any point then they leave. After all experts have provided their analysis, you then analyze all 3 analyses and provide either the consensus solution or your best guess solution. The question is: Create a Kubernetes YAML manifest to deploy a WordPress website using a MySQL database as the backend. Name the WordPress container "wordpress" and the MySQL container "mysql"."""

d_tot_prompt_1 = """Imagine three different Kubernetes experts are answering this question. All experts will write down 1 step of their thinking, then share it with the group. Then all experts will go on to the next step, etc. If any expert realises they're wrong at any point then they leave. The question is: Create a Kubernetes YAML manifest to deploy a WordPress website with a MySQL database as the backend. Ensure the manifest includes the following:
	- Name the WordPress container "wordpress" and the MySQL container "mysql".
	- A Deployment for WordPress with appropriate container images, ports, and environment variables for database configuration.
	- A Deployment for MySQL, configured with PersistentVolumeClaims for data persistence.
	- Services for both WordPress and MySQL to enable connectivity.
	- Use ConfigMaps and Secrets for configuration and sensitive data."""
d_tot_prompt_2 = """Simulate three brilliant, Kubernetes experts collaboratively answering a question. Each one verbosely explains their thought process in real-time, considering the prior explanations of others and openly acknowledging mistakes. At each step, whenever possible, each expert refines and builds upon the thoughts of others, acknowledging their contributions. They continue until there is a definitive answer to the question. The question is: Create a Kubernetes YAML manifest to deploy a WordPress website with a MySQL database as the backend. Ensure the manifest includes the following:
	- Name the WordPress container "wordpress" and the MySQL container "mysql".
	- A Deployment for WordPress with appropriate container images, ports, and environment variables for database configuration.
	- A Deployment for MySQL, configured with PersistentVolumeClaims for data persistence.
	- Services for both WordPress and MySQL to enable connectivity.
	- Use ConfigMaps and Secrets for configuration and sensitive data."""
d_tot_prompt_3 = """Identify and behave as three different experts that are appropriate to answering this question. All experts will write down the step and their thinking about the step, then share it with the group. Then, all experts will go on to the next step, etc. At each step all experts will score their peers response between 1 and 5, 1 meaning it is highly unlikely, and 5 meaning it is highly likely. If any expert is judged to be wrong at any point then they leave. After all experts have provided their analysis, you then analyze all 3 analyses and provide either the consensus solution or your best guess solution. The question is: Create a Kubernetes YAML manifest to deploy a WordPress website with a MySQL database as the backend. Ensure the manifest includes the following:
	- Name the WordPress container "wordpress" and the MySQL container "mysql".
	- A Deployment for WordPress with appropriate container images, ports, and environment variables for database configuration.
	- A Deployment for MySQL, configured with PersistentVolumeClaims for data persistence.
	- Services for both WordPress and MySQL to enable connectivity.
	- Use ConfigMaps and Secrets for configuration and sensitive data."""

## Zero-Shot Prompting
- Baseline system prompt + basic user prompt
- Baseline system prompt + detailed user prompt
- Role-based system prompt + basic user prompt
- Role-Based system prompt + detailed user prompt
- Role-based best practice system prompt + basic user prompt
- Role-based best practice system prompt + detailed user prompt



In [8]:
# gpt-3.5-turbo-0125
directory = './gpt3_5/zero_shot/baseline_system_prompt'
responses = request_openai(bl_system_prompt, bl_user_prompt, model="gpt-3.5-turbo-0125", n=50)
save_response_LLM(directory, responses)

directory = './gpt3_5/zero_shot/baseline_system_prompt_detailed'
responses = request_openai(bl_system_prompt, bl_detailed_user_prompt, model="gpt-3.5-turbo-0125", n=50)
save_response_LLM(directory, responses)

directory = './gpt3_5/zero_shot/role_system_prompt'
responses = request_openai(role_system_prompt, bl_user_prompt, model="gpt-3.5-turbo-0125", n=50)
save_response_LLM(directory, responses)

directory = './gpt3_5/zero_shot/role_system_prompt_detailed'
responses = request_openai(role_system_prompt, bl_detailed_user_prompt, model="gpt-3.5-turbo-0125", n=50)
save_response_LLM(directory, responses)

directory = './gpt3_5/zero_shot/role_best_system_prompt'
responses = request_openai(role_best_system_prompt, bl_user_prompt, model="gpt-3.5-turbo-0125", n=50)
save_response_LLM(directory, responses)

directory = './gpt3_5/zero_shot/role_best_system_prompt_detailed'
responses = request_openai(role_best_system_prompt, bl_detailed_user_prompt, model="gpt-3.5-turbo-0125", n=50)
save_response_LLM(directory, responses)

# GPT-4o-2024-08-06
directory = './gpt4o/zero_shot/baseline_system_prompt'
responses = request_openai(bl_system_prompt, bl_user_prompt, model="gpt-4o-2024-08-06", n=50)
save_response_LLM(directory, responses)

directory = './gpt4o/zero_shot/baseline_system_prompt_detailed'
responses = request_openai(bl_system_prompt, bl_detailed_user_prompt, model="gpt-4o-2024-08-06", n=50)
save_response_LLM(directory, responses)

directory = './gpt4o/zero_shot/role_system_prompt'
responses = request_openai(role_system_prompt, bl_user_prompt, model="gpt-4o-2024-08-06", n=50)
save_response_LLM(directory, responses)

directory = './gpt4o/zero_shot/role_system_prompt_detailed'
responses = request_openai(role_system_prompt, bl_detailed_user_prompt, model="gpt-4o-2024-08-06", n=50)
save_response_LLM(directory, responses)

directory = './gpt4o/zero_shot/role_best_system_prompt'
responses = request_openai(role_best_system_prompt, bl_user_prompt, model="gpt-4o-2024-08-06", n=50)
save_response_LLM(directory, responses)

directory = './gpt4o/zero_shot/role_best_system_prompt_detailed'
responses = request_openai(role_best_system_prompt, bl_detailed_user_prompt, model="gpt-4o-2024-08-06", n=50)
save_response_LLM(directory, responses)

Started requesting LLM
Saved LLM response to: ./gpt3_5/zero_shot/baseline_system_prompt/response-0.yaml
Saved LLM response to: ./gpt3_5/zero_shot/baseline_system_prompt/response-1.yaml
Saved LLM response to: ./gpt3_5/zero_shot/baseline_system_prompt/response-2.yaml
Saved LLM response to: ./gpt3_5/zero_shot/baseline_system_prompt/response-3.yaml
Saved LLM response to: ./gpt3_5/zero_shot/baseline_system_prompt/response-4.yaml
Saved LLM response to: ./gpt3_5/zero_shot/baseline_system_prompt/response-5.yaml
Saved LLM response to: ./gpt3_5/zero_shot/baseline_system_prompt/response-6.yaml
Saved LLM response to: ./gpt3_5/zero_shot/baseline_system_prompt/response-7.yaml
Saved LLM response to: ./gpt3_5/zero_shot/baseline_system_prompt/response-8.yaml
Saved LLM response to: ./gpt3_5/zero_shot/baseline_system_prompt/response-9.yaml
Saved LLM response to: ./gpt3_5/zero_shot/baseline_system_prompt/response-10.yaml
Saved LLM response to: ./gpt3_5/zero_shot/baseline_system_prompt/response-11.yaml
Sav

In [9]:
# gpt-4-turbo-2024-04-09
directory = './gpt4/zero_shot/baseline_system_prompt'
responses = request_openai(bl_system_prompt, bl_user_prompt, model="gpt-4-turbo-2024-04-09", n=50)
save_response_LLM(directory, responses)

directory = './gpt4/zero_shot/baseline_system_prompt_detailed'
responses = request_openai(bl_system_prompt, bl_detailed_user_prompt, model="gpt-4-turbo-2024-04-09", n=50)
save_response_LLM(directory, responses)

directory = './gpt4/zero_shot/role_system_prompt'
responses = request_openai(role_system_prompt, bl_user_prompt, model="gpt-4-turbo-2024-04-09", n=50)
save_response_LLM(directory, responses)

directory = './gpt4/zero_shot/role_system_prompt_detailed'
responses = request_openai(role_system_prompt, bl_detailed_user_prompt, model="gpt-4-turbo-2024-04-09", n=50)
save_response_LLM(directory, responses)

directory = './gpt4/zero_shot/role_best_system_prompt'
responses = request_openai(role_best_system_prompt, bl_user_prompt, model="gpt-4-turbo-2024-04-09", n=50)
save_response_LLM(directory, responses)

directory = './gpt4/zero_shot/role_best_system_prompt_detailed'
responses = request_openai(role_best_system_prompt, bl_detailed_user_prompt, model="gpt-4-turbo-2024-04-09", n=50)
save_response_LLM(directory, responses)

Started requesting LLM
Saved LLM response to: ./gpt4/zero_shot/baseline_system_prompt/response-0.yaml
Saved LLM response to: ./gpt4/zero_shot/baseline_system_prompt/response-1.yaml
Saved LLM response to: ./gpt4/zero_shot/baseline_system_prompt/response-2.yaml
Saved LLM response to: ./gpt4/zero_shot/baseline_system_prompt/response-3.yaml
Saved LLM response to: ./gpt4/zero_shot/baseline_system_prompt/response-4.yaml
Saved LLM response to: ./gpt4/zero_shot/baseline_system_prompt/response-5.yaml
Saved LLM response to: ./gpt4/zero_shot/baseline_system_prompt/response-6.yaml
Saved LLM response to: ./gpt4/zero_shot/baseline_system_prompt/response-7.yaml
Saved LLM response to: ./gpt4/zero_shot/baseline_system_prompt/response-8.yaml
Saved LLM response to: ./gpt4/zero_shot/baseline_system_prompt/response-9.yaml
Saved LLM response to: ./gpt4/zero_shot/baseline_system_prompt/response-10.yaml
Saved LLM response to: ./gpt4/zero_shot/baseline_system_prompt/response-11.yaml
Saved LLM response to: ./gp

## Zero-Shot CoT Prompting
- Basic user prompt + Human-designed reasoning sentence
- Detailed user prompt + Human-designed reasoning sentence
- Basic user prompt + APE reasoning sentence
- Detailed user prompt + APE reasoning sentence


In [10]:
# gpt-3.5-turbo-0125
directory = './gpt3_5/cot/human_prompt'
responses = request_openai(None, reasoning_prompt, model="gpt-3.5-turbo-0125", n=50)
second_request_openai(directory, reasoning_prompt, extraction_prompt, responses, model="gpt-3.5-turbo-0125")

directory = './gpt3_5/cot/human_prompt_detailed'
responses = request_openai(None, d_reasoning_prompt, model="gpt-3.5-turbo-0125", n=50)
second_request_openai(directory, d_reasoning_prompt, extraction_prompt, responses, model="gpt-3.5-turbo-0125")

directory = './gpt3_5/cot/ape_prompt'
responses = request_openai(None, ape_reasoning_prompt, model="gpt-3.5-turbo-0125", n=50)
second_request_openai(directory, ape_reasoning_prompt, extraction_prompt, responses, model="gpt-3.5-turbo-0125")

directory = './gpt3_5/cot/ape_prompt_detailed'
responses = request_openai(None, ape_d_reasoning_prompt, model="gpt-3.5-turbo-0125", n=50)
second_request_openai(directory, ape_d_reasoning_prompt, extraction_prompt, responses, model="gpt-3.5-turbo-0125")

Started requesting LLM
Requesting second time LLM for response: 0
Saved LLM response to: ./gpt3_5/cot/human_prompt/response-0.yaml
Requesting second time LLM for response: 1
Saved LLM response to: ./gpt3_5/cot/human_prompt/response-1.yaml
Requesting second time LLM for response: 2
Saved LLM response to: ./gpt3_5/cot/human_prompt/response-2.yaml
Requesting second time LLM for response: 3
Saved LLM response to: ./gpt3_5/cot/human_prompt/response-3.yaml
Requesting second time LLM for response: 4
Saved LLM response to: ./gpt3_5/cot/human_prompt/response-4.yaml
Requesting second time LLM for response: 5
Saved LLM response to: ./gpt3_5/cot/human_prompt/response-5.yaml
Requesting second time LLM for response: 6
Saved LLM response to: ./gpt3_5/cot/human_prompt/response-6.yaml
Requesting second time LLM for response: 7
Saved LLM response to: ./gpt3_5/cot/human_prompt/response-7.yaml
Requesting second time LLM for response: 8
Saved LLM response to: ./gpt3_5/cot/human_prompt/response-8.yaml
Reque

In [11]:
# gpt-4o-2024-08-06
directory = './gpt4o/cot/human_prompt'
responses = request_openai(None, reasoning_prompt, model="gpt-4o-2024-08-06", n=50)
second_request_openai(directory, reasoning_prompt, extraction_prompt, responses, model="gpt-4o-2024-08-06")

directory = './gpt4o/cot/human_prompt_detailed'
responses = request_openai(None, d_reasoning_prompt, model="gpt-4o-2024-08-06", n=50)
second_request_openai(directory, d_reasoning_prompt, extraction_prompt, responses, model="gpt-4o-2024-08-06")

directory = './gpt4o/cot/ape_prompt'
responses = request_openai(None, ape_reasoning_prompt, model="gpt-4o-2024-08-06", n=50)
second_request_openai(directory, ape_reasoning_prompt, extraction_prompt, responses, model="gpt-4o-2024-08-06")

directory = './gpt4o/cot/ape_prompt_detailed'
responses = request_openai(None, ape_d_reasoning_prompt, model="gpt-4o-2024-08-06", n=50)
second_request_openai(directory, ape_d_reasoning_prompt, extraction_prompt, responses, model="gpt-4o-2024-08-06")

Started requesting LLM
Requesting second time LLM for response: 0
Saved LLM response to: ./gpt4o/cot/human_prompt/response-0.yaml
Requesting second time LLM for response: 1
Saved LLM response to: ./gpt4o/cot/human_prompt/response-1.yaml
Requesting second time LLM for response: 2
Saved LLM response to: ./gpt4o/cot/human_prompt/response-2.yaml
Requesting second time LLM for response: 3
Saved LLM response to: ./gpt4o/cot/human_prompt/response-3.yaml
Requesting second time LLM for response: 4
Saved LLM response to: ./gpt4o/cot/human_prompt/response-4.yaml
Requesting second time LLM for response: 5
Saved LLM response to: ./gpt4o/cot/human_prompt/response-5.yaml
Requesting second time LLM for response: 6
Saved LLM response to: ./gpt4o/cot/human_prompt/response-6.yaml
Requesting second time LLM for response: 7
Saved LLM response to: ./gpt4o/cot/human_prompt/response-7.yaml
Requesting second time LLM for response: 8
Saved LLM response to: ./gpt4o/cot/human_prompt/response-8.yaml
Requesting sec

In [12]:
# gpt-4-turbo-2024-04-09
directory = './gpt4/cot/human_prompt'
responses = request_openai(None, reasoning_prompt, model="gpt-4-turbo-2024-04-09", n=50)
second_request_openai(directory, reasoning_prompt, extraction_prompt, responses, model="gpt-4-turbo-2024-04-09")

directory = './gpt4/cot/human_prompt_detailed'
responses = request_openai(None, d_reasoning_prompt, model="gpt-4-turbo-2024-04-09", n=50)
second_request_openai(directory, d_reasoning_prompt, extraction_prompt, responses, model="gpt-4-turbo-2024-04-09")

directory = './gpt4/cot/ape_prompt'
responses = request_openai(None, ape_reasoning_prompt, model="gpt-4-turbo-2024-04-09", n=50)
second_request_openai(directory, ape_reasoning_prompt, extraction_prompt, responses, model="gpt-4-turbo-2024-04-09")

directory = './gpt4/cot/ape_prompt_detailed'
responses = request_openai(None, ape_d_reasoning_prompt, model="gpt-4-turbo-2024-04-09", n=50)
second_request_openai(directory, ape_d_reasoning_prompt, extraction_prompt, responses, model="gpt-4-turbo-2024-04-09")

Started requesting LLM
Requesting second time LLM for response: 0
Saved LLM response to: ./gpt4/cot/human_prompt/response-0.yaml
Requesting second time LLM for response: 1
Saved LLM response to: ./gpt4/cot/human_prompt/response-1.yaml
Requesting second time LLM for response: 2
Saved LLM response to: ./gpt4/cot/human_prompt/response-2.yaml
Requesting second time LLM for response: 3
Saved LLM response to: ./gpt4/cot/human_prompt/response-3.yaml
Requesting second time LLM for response: 4
Saved LLM response to: ./gpt4/cot/human_prompt/response-4.yaml
Requesting second time LLM for response: 5
Saved LLM response to: ./gpt4/cot/human_prompt/response-5.yaml
Requesting second time LLM for response: 6
Saved LLM response to: ./gpt4/cot/human_prompt/response-6.yaml
Requesting second time LLM for response: 7
Saved LLM response to: ./gpt4/cot/human_prompt/response-7.yaml
Requesting second time LLM for response: 8
Saved LLM response to: ./gpt4/cot/human_prompt/response-8.yaml
Requesting second time 

# Meta prompting
- Meta system prompt + basic user prompt
- Meta system prompt + detailed user prompt
- Meta meta prompt to enhance basic user prompt (+ baseline system prompt)
- Meta meta prompt to enhance detailed user prompt (+ baseline system prompt)

In [13]:
# gpt-3.5-turbo-0125
directory = './gpt3_5/meta/meta_system_prompt'
responses = request_openai(meta_system_prompt, bl_user_prompt, model="gpt-3.5-turbo-0125", n=50)
save_response_LLM(directory, responses)

directory = './gpt3_5/meta/meta_system_prompt_detailed'
responses = request_openai(meta_system_prompt, bl_detailed_user_prompt, model="gpt-3.5-turbo-0125", n=50)
save_response_LLM(directory, responses)

directory = './gpt3_5/meta/meta_meta_prompt'
responses = request_openai(None, meta_meta_prompt, model="gpt-3.5-turbo-0125", n=50)
meta_request_openai(directory, bl_system_prompt, responses, model="gpt-3.5-turbo-0125")

directory = './gpt3_5/meta/meta_meta_prompt_detailed'
responses = request_openai(None, d_meta_meta_prompt, model="gpt-3.5-turbo-0125", n=50)
meta_request_openai(directory, bl_system_prompt, responses, model="gpt-3.5-turbo-0125")

Started requesting LLM
Saved LLM response to: ./gpt3_5/meta/meta_system_prompt/response-0.yaml
Saved LLM response to: ./gpt3_5/meta/meta_system_prompt/response-1.yaml
Saved LLM response to: ./gpt3_5/meta/meta_system_prompt/response-2.yaml
Saved LLM response to: ./gpt3_5/meta/meta_system_prompt/response-3.yaml
Saved LLM response to: ./gpt3_5/meta/meta_system_prompt/response-4.yaml
Saved LLM response to: ./gpt3_5/meta/meta_system_prompt/response-5.yaml
Saved LLM response to: ./gpt3_5/meta/meta_system_prompt/response-6.yaml
Saved LLM response to: ./gpt3_5/meta/meta_system_prompt/response-7.yaml
Saved LLM response to: ./gpt3_5/meta/meta_system_prompt/response-8.yaml
Saved LLM response to: ./gpt3_5/meta/meta_system_prompt/response-9.yaml
Saved LLM response to: ./gpt3_5/meta/meta_system_prompt/response-10.yaml
Saved LLM response to: ./gpt3_5/meta/meta_system_prompt/response-11.yaml
Saved LLM response to: ./gpt3_5/meta/meta_system_prompt/response-12.yaml
Saved LLM response to: ./gpt3_5/meta/m

In [14]:
# gpt-4o-2024-08-06
directory = './gpt4o/meta/meta_system_prompt'
responses = request_openai(meta_system_prompt, bl_user_prompt, model="gpt-4o-2024-08-06", n=50)
save_response_LLM(directory, responses)

directory = './gpt4o/meta/meta_system_prompt_detailed'
responses = request_openai(meta_system_prompt, bl_detailed_user_prompt, model="gpt-4o-2024-08-06", n=50)
save_response_LLM(directory, responses)

directory = './gpt4o/meta/meta_meta_prompt'
responses = request_openai(None, meta_meta_prompt, model="gpt-4o-2024-08-06", n=50)
meta_request_openai(directory, bl_system_prompt, responses, model="gpt-4o-2024-08-06")

directory = './gpt4o/meta/meta_meta_prompt_detailed'
responses = request_openai(None, d_meta_meta_prompt, model="gpt-4o-2024-08-06", n=50)
meta_request_openai(directory, bl_system_prompt, responses, model="gpt-4o-2024-08-06")

Started requesting LLM
Saved LLM response to: ./gpt4o/meta/meta_system_prompt/response-0.yaml
Saved LLM response to: ./gpt4o/meta/meta_system_prompt/response-1.yaml
Saved LLM response to: ./gpt4o/meta/meta_system_prompt/response-2.yaml
Saved LLM response to: ./gpt4o/meta/meta_system_prompt/response-3.yaml
Saved LLM response to: ./gpt4o/meta/meta_system_prompt/response-4.yaml
Saved LLM response to: ./gpt4o/meta/meta_system_prompt/response-5.yaml
Saved LLM response to: ./gpt4o/meta/meta_system_prompt/response-6.yaml
Saved LLM response to: ./gpt4o/meta/meta_system_prompt/response-7.yaml
Saved LLM response to: ./gpt4o/meta/meta_system_prompt/response-8.yaml
Saved LLM response to: ./gpt4o/meta/meta_system_prompt/response-9.yaml
Saved LLM response to: ./gpt4o/meta/meta_system_prompt/response-10.yaml
Saved LLM response to: ./gpt4o/meta/meta_system_prompt/response-11.yaml
Saved LLM response to: ./gpt4o/meta/meta_system_prompt/response-12.yaml
Saved LLM response to: ./gpt4o/meta/meta_system_pro

In [15]:
# gpt-4-turbo-2024-04-09
directory = './gpt4/meta/meta_system_prompt'
responses = request_openai(meta_system_prompt, bl_user_prompt, model="gpt-4-turbo-2024-04-09", n=50)
save_response_LLM(directory, responses)

directory = './gpt4/meta/meta_system_prompt_detailed'
responses = request_openai(meta_system_prompt, bl_detailed_user_prompt, model="gpt-4-turbo-2024-04-09", n=50)
save_response_LLM(directory, responses)

directory = './gpt4/meta/meta_meta_prompt'
responses = request_openai(None, meta_meta_prompt, model="gpt-4-turbo-2024-04-09", n=50)
meta_request_openai(directory, bl_system_prompt, responses, model="gpt-4-turbo-2024-04-09")

directory = './gpt4/meta/meta_meta_prompt_detailed'
responses = request_openai(None, d_meta_meta_prompt, model="gpt-4-turbo-2024-04-09", n=50)
meta_request_openai(directory, bl_system_prompt, responses, model="gpt-4-turbo-2024-04-09")

Started requesting LLM
Saved LLM response to: ./gpt4/meta/meta_system_prompt/response-0.yaml
Saved LLM response to: ./gpt4/meta/meta_system_prompt/response-1.yaml
Saved LLM response to: ./gpt4/meta/meta_system_prompt/response-2.yaml
Saved LLM response to: ./gpt4/meta/meta_system_prompt/response-3.yaml
Saved LLM response to: ./gpt4/meta/meta_system_prompt/response-4.yaml
Saved LLM response to: ./gpt4/meta/meta_system_prompt/response-5.yaml
Saved LLM response to: ./gpt4/meta/meta_system_prompt/response-6.yaml
Saved LLM response to: ./gpt4/meta/meta_system_prompt/response-7.yaml
Saved LLM response to: ./gpt4/meta/meta_system_prompt/response-8.yaml
Saved LLM response to: ./gpt4/meta/meta_system_prompt/response-9.yaml
Saved LLM response to: ./gpt4/meta/meta_system_prompt/response-10.yaml
Saved LLM response to: ./gpt4/meta/meta_system_prompt/response-11.yaml
Saved LLM response to: ./gpt4/meta/meta_system_prompt/response-12.yaml
Saved LLM response to: ./gpt4/meta/meta_system_prompt/response-1

# ToT prompting
- ToT prompt 1 + basic user prompt
- ToT prompt 2 + basic user prompt
- ToT prompt 3 + basic user prompt
- ToT prompt 1 + detailed user prompt
- ToT prompt 2 + detailed user prompt
- ToT prompt 3 + detailed user prompt

In [16]:
# gpt-3.5-turbo-0125
directory = './gpt3_5/tot/tot_prompt_1'
responses = request_openai(bl_system_prompt, tot_prompt_1, model="gpt-3.5-turbo-0125", n=50)
save_response_LLM(directory, responses)

directory = './gpt3_5/tot/tot_prompt_2'
responses = request_openai(bl_system_prompt, tot_prompt_2, model="gpt-3.5-turbo-0125", n=50)
save_response_LLM(directory, responses)

directory = './gpt3_5/tot/tot_prompt_3'
responses = request_openai(bl_system_prompt, tot_prompt_3, model="gpt-3.5-turbo-0125", n=50)
save_response_LLM(directory, responses)

directory = './gpt3_5/tot/tot_detailed_prompt_1'
responses = request_openai(bl_system_prompt, d_tot_prompt_1, model="gpt-3.5-turbo-0125", n=50)
save_response_LLM(directory, responses)

directory = './gpt3_5/tot/tot_detailed_prompt_2'
responses = request_openai(bl_system_prompt, d_tot_prompt_2, model="gpt-3.5-turbo-0125", n=50)
save_response_LLM(directory, responses)

directory = './gpt3_5/tot/tot_detailed_prompt_3'
responses = request_openai(bl_system_prompt, d_tot_prompt_3, model="gpt-3.5-turbo-0125", n=50)
save_response_LLM(directory, responses)

Started requesting LLM
Saved LLM response to: ./gpt3_5/tot/tot_prompt_1/response-0.yaml
Saved LLM response to: ./gpt3_5/tot/tot_prompt_1/response-1.yaml
Saved LLM response to: ./gpt3_5/tot/tot_prompt_1/response-2.yaml
Saved LLM response to: ./gpt3_5/tot/tot_prompt_1/response-3.yaml
Saved LLM response to: ./gpt3_5/tot/tot_prompt_1/response-4.yaml
Saved LLM response to: ./gpt3_5/tot/tot_prompt_1/response-5.yaml
Saved LLM response to: ./gpt3_5/tot/tot_prompt_1/response-6.yaml
Saved LLM response to: ./gpt3_5/tot/tot_prompt_1/response-7.yaml
Saved LLM response to: ./gpt3_5/tot/tot_prompt_1/response-8.yaml
Saved LLM response to: ./gpt3_5/tot/tot_prompt_1/response-9.yaml
Saved LLM response to: ./gpt3_5/tot/tot_prompt_1/response-10.yaml
Saved LLM response to: ./gpt3_5/tot/tot_prompt_1/response-11.yaml
Saved LLM response to: ./gpt3_5/tot/tot_prompt_1/response-12.yaml
Saved LLM response to: ./gpt3_5/tot/tot_prompt_1/response-13.yaml
Saved LLM response to: ./gpt3_5/tot/tot_prompt_1/response-14.ya

In [17]:
# gpt-4o-2024-08-06
directory = './gpt4o/tot/tot_prompt_1'
responses = request_openai(bl_system_prompt, tot_prompt_1, model="gpt-4o-2024-08-06", n=50)
save_response_LLM(directory, responses)

directory = './gpt4o/tot/tot_prompt_2'
responses = request_openai(bl_system_prompt, tot_prompt_2, model="gpt-4o-2024-08-06", n=50)
save_response_LLM(directory, responses)

directory = './gpt4o/tot/tot_prompt_3'
responses = request_openai(bl_system_prompt, tot_prompt_3, model="gpt-4o-2024-08-06", n=50)
save_response_LLM(directory, responses)

directory = './gpt4o/tot/tot_detailed_prompt_1'
responses = request_openai(bl_system_prompt, d_tot_prompt_1, model="gpt-4o-2024-08-06", n=50)
save_response_LLM(directory, responses)

directory = './gpt4o/tot/tot_detailed_prompt_2'
responses = request_openai(bl_system_prompt, d_tot_prompt_2, model="gpt-4o-2024-08-06", n=50)
save_response_LLM(directory, responses)

directory = './gpt4o/tot/tot_detailed_prompt_3'
responses = request_openai(bl_system_prompt, d_tot_prompt_3, model="gpt-4o-2024-08-06", n=50)
save_response_LLM(directory, responses)

Started requesting LLM
Saved LLM response to: ./gpt4o/tot/tot_prompt_1/response-0.yaml
Saved LLM response to: ./gpt4o/tot/tot_prompt_1/response-1.yaml
Saved LLM response to: ./gpt4o/tot/tot_prompt_1/response-2.yaml
Saved LLM response to: ./gpt4o/tot/tot_prompt_1/response-3.yaml
Saved LLM response to: ./gpt4o/tot/tot_prompt_1/response-4.yaml
Saved LLM response to: ./gpt4o/tot/tot_prompt_1/response-5.yaml
Saved LLM response to: ./gpt4o/tot/tot_prompt_1/response-6.yaml
Saved LLM response to: ./gpt4o/tot/tot_prompt_1/response-7.yaml
Saved LLM response to: ./gpt4o/tot/tot_prompt_1/response-8.yaml
Saved LLM response to: ./gpt4o/tot/tot_prompt_1/response-9.yaml
Saved LLM response to: ./gpt4o/tot/tot_prompt_1/response-10.yaml
Saved LLM response to: ./gpt4o/tot/tot_prompt_1/response-11.yaml
Saved LLM response to: ./gpt4o/tot/tot_prompt_1/response-12.yaml
Saved LLM response to: ./gpt4o/tot/tot_prompt_1/response-13.yaml
Saved LLM response to: ./gpt4o/tot/tot_prompt_1/response-14.yaml
Saved LLM re

In [18]:
# gpt-4-turbo-2024-04-09
directory = './gpt4/tot/tot_prompt_1'
responses = request_openai(bl_system_prompt, tot_prompt_1, model="gpt-4-turbo-2024-04-09", n=50)
save_response_LLM(directory, responses)

directory = './gpt4/tot/tot_prompt_2'
responses = request_openai(bl_system_prompt, tot_prompt_2, model="gpt-4-turbo-2024-04-09", n=50)
save_response_LLM(directory, responses)

directory = './gpt4/tot/tot_prompt_3'
responses = request_openai(bl_system_prompt, tot_prompt_3, model="gpt-4-turbo-2024-04-09", n=50)
save_response_LLM(directory, responses)

directory = './gpt4/tot/tot_detailed_prompt_1'
responses = request_openai(bl_system_prompt, d_tot_prompt_1, model="gpt-4-turbo-2024-04-09", n=50)
save_response_LLM(directory, responses)

directory = './gpt4/tot/tot_detailed_prompt_2'
responses = request_openai(bl_system_prompt, d_tot_prompt_2, model="gpt-4-turbo-2024-04-09", n=50)
save_response_LLM(directory, responses)

directory = './gpt4/tot/tot_detailed_prompt_3'
responses = request_openai(bl_system_prompt, d_tot_prompt_3, model="gpt-4-turbo-2024-04-09", n=50)
save_response_LLM(directory, responses)

Started requesting LLM
Saved LLM response to: ./gpt4/tot/tot_prompt_1/response-0.yaml
Saved LLM response to: ./gpt4/tot/tot_prompt_1/response-1.yaml
Saved LLM response to: ./gpt4/tot/tot_prompt_1/response-2.yaml
Saved LLM response to: ./gpt4/tot/tot_prompt_1/response-3.yaml
Saved LLM response to: ./gpt4/tot/tot_prompt_1/response-4.yaml
Saved LLM response to: ./gpt4/tot/tot_prompt_1/response-5.yaml
Saved LLM response to: ./gpt4/tot/tot_prompt_1/response-6.yaml
Saved LLM response to: ./gpt4/tot/tot_prompt_1/response-7.yaml
Saved LLM response to: ./gpt4/tot/tot_prompt_1/response-8.yaml
Saved LLM response to: ./gpt4/tot/tot_prompt_1/response-9.yaml
Saved LLM response to: ./gpt4/tot/tot_prompt_1/response-10.yaml
Saved LLM response to: ./gpt4/tot/tot_prompt_1/response-11.yaml
Saved LLM response to: ./gpt4/tot/tot_prompt_1/response-12.yaml
Saved LLM response to: ./gpt4/tot/tot_prompt_1/response-13.yaml
Saved LLM response to: ./gpt4/tot/tot_prompt_1/response-14.yaml
Saved LLM response to: ./gp